# Cine21 Crawling

## 1. Import libraries

In [2]:
import time
import pandas as pd

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

## 2. Connect to Cine21 website

In [3]:
start_time = time.time()

from selenium import webdriver

driver = webdriver.Chrome("./chromedriver")

# 씨네21 영화 검색 페이지 접속 (''검색 결과)
driver.get("http://www.cine21.com/")

print("--- %s seconds ---" % (time.time() - start_time))

--- 17.34216046333313 seconds ---


## 3. Prepare Data
네이버 3년치 데이터 가져온 뒤 취합

In [4]:
naver_df_2019 = pd.read_csv('C:/Users/Danbee/Desktop/luckyseven/naver_data/naver_2019_poster.csv', encoding='UTF8')
naver_df_2020 = pd.read_csv('C:/Users/Danbee/Desktop/luckyseven/naver_data/naver_2020_poster.csv', encoding='UTF8')
naver_df_2021 = pd.read_csv('C:/Users/Danbee/Desktop/luckyseven/naver_data/naver_2021_poster.csv', encoding='UTF8')

In [5]:
print(len(naver_df_2019)) # 722 vs 850
print(len(naver_df_2020)) # 647 vs 730
print(len(naver_df_2021)) # 549 vs 728

# 누락된 데이터 수
print("2019년에서 누락된 데이터 수:", 850-722)
print("2020년에서 누락된 데이터 수:", 730-647)
print("2021년에서 누락된 데이터 수:", 728-549)

# 3년 전체
print("2019~2021년에서 누락된 데이터 수:", (850+730+728)-(722+647+549))

722
647
549
2019년에서 누락된 데이터 수: 128
2020년에서 누락된 데이터 수: 83
2021년에서 누락된 데이터 수: 179
2019~2021년에서 누락된 데이터 수: 390


In [6]:
# 모아온 3년 전체 데이터 수 = 722+647+549 = 1918
df = pd.concat([naver_df_2019, naver_df_2020, naver_df_2021], ignore_index=True)

## 4. find the match and get urls
네이버 영화 리스트를 받아서 씨네21에서 title로 검색하여 나온 결과 페이지의 1페이지를 차례로 보면서
title과 director가 매칭되면 해당 영화의 상세페이지로 가는 url을 리턴한다.

In [55]:
def find_matching_movie(title_director_list):
    
    title = title_director_list[0]
    director = title_director_list[1]
    
    # 검색창: input#search_q.input_search
    search_box = driver.find_element_by_css_selector("input#search_q.input_search")
    # 검색버튼: a.btn_search
    search_btn = driver.find_element_by_css_selector("#total_search_form > fieldset > a")
    
#     time.sleep(2)
    
    # 홈페이지에서 제목으로 검색
    search_box.send_keys(title)
    search_btn.click()
    
#     time.sleep(2)

    # 영화 탭 버튼 눌러서 영화에 대한 검색 결과만 보기
    movie_tab_btn = driver.find_element_by_css_selector("#snb_t > li:nth-child(2) > a")
    movie_tab_btn.click()
    
    time.sleep(2)

    # 일단 해당 영화 제목으로 검색해서 검색 결과가 있는 지 확인
    try:
        # 결과가 하나라도 있는 경우
        # 검색 결과 수 보기
        num_of_movies = driver.find_element_by_css_selector("#content > div.culm2_area > div.culm2_l > h2 > span").text
        num_of_movies = int(num_of_movies.replace('영화', '').replace('(', '').replace(')', '').replace(' ', ''))
#         print(f'씨네21에서 {title}에 대해 찾은 검색 결과는 {num_of_movies}건 입니다.')

#         time.sleep(2)
        
        # 영화 검색 결과 페이지에서
        # 일단 1페이지에 내가 찾는 있다고 가정
        # 영화 제목, 감독 일치하면 해당 페이지로 들어가서 정보 가져오기
        
        for i in range(1, 11):

            titles = driver.find_elements_by_css_selector(f"#content > div.culm2_area > div.culm2_l > ul.mov_list > li:nth-child({i}) > p.name > a")
            href = titles[0].get_attribute('href')
            
            try:
                directors = driver.find_elements_by_css_selector(f"#content > div.culm2_area > div.culm2_l > ul.mov_list > li:nth-child({i}) > p:nth-child(4) > a:nth-child(2)")
                
#                 print('제목:', title, titles[0].text)
#                 print('감독:', director, directors[0].text)

    #             time.sleep(2)

                if (title == titles[0].text) and (director.replace('.', '') == directors[0].text.replace('.','')):
                    return href

                else:
#                     print(f'{title}와/과 제목과 감독이 일치하는 영화가 이 페이지에 없습니다.')
                    return('매칭 영화 없음')
    #                 continue
    
            except IndexError:
                pass
#                 print(f"{title}에 대한 감독 정보가 없습니다")
    
    except NoSuchElementException:
#         print(f"{title}에 대한 검색 결과가 없습니다.")
        return('검색 결과 없음')

### find_matching_movie function testing
테스팅 데이터 준비

In [57]:
test_list = list(map(list, zip(df['title'], df['director'])))[:5]
test_list.append(['글래스', 'M 나이트 샤말란'])
test_list

[['0.0MHz', '유선동'],
 ['1 더하기 3', '프룻 첸'],
 ['10 미니츠 곤', '브라이언 A 밀러'],
 ['100일 동안 100가지로 100퍼센트 행복찾기', '플로리안 데이비드 핏츠'],
 ['10년', '하야카와 치에, 키노시타 유스케, 츠노 메구미'],
 ['글래스', 'M 나이트 샤말란']]

In [58]:
start_time = time.time()

urls_from_cine = []

for i in test_list:
    result = find_matching_movie(i)
    urls_from_cine.append(result)
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 39.1094331741333 seconds ---


In [61]:
urls_from_cine # 371 rows 있어야 함 (value는 url, 검색 결과 없음, 매칭 영화 없음 셋 중 하나)

['http://www.cine21.com/movie/info/?movie_id=52988',
 '검색 결과 없음',
 'http://www.cine21.com/movie/info/?movie_id=55539',
 'http://www.cine21.com/movie/info/?movie_id=55031',
 '매칭 영화 없음',
 'http://www.cine21.com/movie/info/?movie_id=53710']

### parmap 코드는 커널이 계속 돌기만 하고 진행이 안됨

# https://dailyheumsi.tistory.com/105
# https://pypi.org/project/parmap/

import numpy as np
import multiprocessing
import parmap

start_time = time.time()

# num_cores = multiprocessing.cpu_count() # 4
num_cores = 2

splited_data = np.array_split(test_list, num_cores)

result = parmap.map(find_matching_movie, splited_data, pm_pbar=True, pm_processes=num_cores)

print("--- %s seconds ---" % (time.time() - start_time))

## 얘도 마찬가지로 계속 돌기만 하고 진행이 안 됨

https://beomi.github.io/2017/07/05/HowToMakeWebCrawler-with-Multiprocess/

from multiprocessing import Pool

start_time = time.time()

pool = Pool(processes=4)
pool.map(find_matching_movie, test_list)

print("--- %s seconds ---" % (time.time() - start-time))

## 네이버 영화 데이터에서 제목과 감독 이름 매칭해서 씨네21 사이트에서 매칭되는 영화 찾아 해당 영화의 상세페이지로 연결되는 url 리스트 얻어오기

In [62]:
naver_list = list(map(list, zip(df['title'], df['director'])))
len(naver_list)

1918

In [63]:
start_time = time.time()

urls_from_cine = []

for i in naver_list:
    print(f"현재 {naver_list.index(i)+1}번째 영화를 찾고 있습니다.")
    result = find_matching_movie(i)
    urls_from_cine.append(result)
    
print("--- %s seconds ---" % (time.time() - start_time))

현재 1번째 영화를 찾고 있습니다.
현재 2번째 영화를 찾고 있습니다.
현재 3번째 영화를 찾고 있습니다.
현재 4번째 영화를 찾고 있습니다.
현재 5번째 영화를 찾고 있습니다.
현재 6번째 영화를 찾고 있습니다.
--- 52.1204469203949 seconds ---


In [64]:
urls_from_cine # 1918 rows 있어야 함 (value는 url, 검색 결과 없음, 매칭 영화 없음 셋 중 하나)

['http://www.cine21.com/movie/info/?movie_id=52988',
 '검색 결과 없음',
 'http://www.cine21.com/movie/info/?movie_id=55539',
 'http://www.cine21.com/movie/info/?movie_id=55031',
 '매칭 영화 없음',
 'http://www.cine21.com/movie/info/?movie_id=53710']